Case: Multi‑period Real‑Estate Co‑Investment

A developer can buy fractional shares of three projects—an office tower (O), a hotel (H), and a shopping center (S). Buying a share of any project commits the firm to fund four staged investments (now, year 1, year 2, year 3) in proportion to its share. Each project also has a total net present value (NPV) for a 100% stake; a partial stake earns that same fraction of NPV.

The firm has separate capital budgets available at each time point (now: $25M; year 1: $20M; year 2: $20M; year 3: $15M). Unused funds at one time do not carry forward.

Investment requirements per project (per 100% stake) and total NPV:

Project	Now	Y1	Y2	Y3	NPV
Office (O)	40	60	90	10	45
Hotel (H)	80	80	80	70	70
Shopping (S)	90	50	20	60	50

(All values in $ millions.)

Goal: Choose shares to maximize total NPV, subject to the per‑period capital limits.

In [7]:
%pip install pulp
import pulp


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
def build_model():
    #---Parameters---
    projects=['Office','Hotel','Shopping']
    T=[0,1,2,3] #Time points
    
    # Investment needs per 100% stake (rows = t, cols = project), in $M
    #            O    H    S
    
    A = {
        0: {"Office": 40, "Hotel": 80, "Shopping": 90},
        1: {"Office": 60, "Hotel": 80, "Shopping": 50},
        2: {"Office": 90, "Hotel": 80, "Shopping": 20},
        3: {"Office": 10, "Hotel": 70, "Shopping": 60},
    }
    
    # Per-period budgets B_t (use-it-or-lose-it), in $M
    B = {0: 25, 1: 20, 2: 20, 3: 15}
    
     # NPV per 100% stake, in $M
    profit = {"Office": 45, "Hotel": 70, "Shopping": 50}
    
    #---Model---
    m=pulp.LpProblem('THinkBig_Capital_Allocation',pulp.LpMaximize)
    
    # Decision variables: fraction invested in each project (0..1)
    x = {p: pulp.LpVariable(f"x_{p}", lowBound=0, upBound=1, cat="Continuous") for p in projects}
    
    # Objective: maximize total NPV
    m += pulp.lpSum(profit[p] * x[p] for p in projects), "Total_NPV"
    
    #Per-period capital constraints
    for t in T:
        m += pulp.lpSum(A[t][p] * x[p] for p in projects) <= B[t], f"Budget_t{t}"
        
        return m,x,A,B,profit

In [20]:
def solve_and_report():
    m, x, A, B, profit = build_model()
    m.solve(pulp.PULP_CBC_CMD(msg=False))

    status = pulp.LpStatus[m.status]
    obj = pulp.value(m.objective)  # in $M

    print(f"Status: {status}")
    for p in x:
        print(f"{p:9s} share: {x[p].varValue:.6f}")
    print(f"Total NPV: ${obj:,.6f}M")

    # Diagnostics: per-period usage and slack
    print("\nPer-period capital usage:")
    for t in sorted(B):
        used = sum(A[t][p] * x[p].varValue for p in x)
        slack = B[t] - used
        print(f"  t={t}: used {used:8.4f} / {B[t]:.4f}   slack={slack:7.4f}  ($M)")


if __name__ == "__main__":
    solve_and_report()

Status: Optimal
Office    share: 0.625000
Hotel     share: 0.000000
Shopping  share: 0.000000
Total NPV: $28.125000M

Per-period capital usage:
  t=0: used  25.0000 / 25.0000   slack= 0.0000  ($M)
  t=1: used  37.5000 / 20.0000   slack=-17.5000  ($M)
  t=2: used  56.2500 / 20.0000   slack=-36.2500  ($M)
  t=3: used   6.2500 / 15.0000   slack= 8.7500  ($M)
